In [ ]:
# Filter data for Kansas City Chiefs games
chiefs_games_home = last_5_years_df[last_5_years_df['team_home_abbr'] == 'KC']
chiefs_games_away = last_5_years_df[last_5_years_df['team_away_abbr'] == 'KC']
chiefs_games = pd.concat([chiefs_games_home, chiefs_games_away])

# Filter games where Kansas City Chiefs won
chiefs_wins_home = last_5_years_df[(last_5_years_df['team_home_abbr'] == 'KC') & (last_5_years_df['score_home'] > last_5_years_df['score_away'])]
chiefs_wins_away = last_5_years_df[(last_5_years_df['team_away_abbr'] == 'KC') & (last_5_years_df['score_away'] > last_5_years_df['score_home'])]
# Filter games where Kansas City Chiefs lost
chiefs_losses_home = last_5_years_df[(last_5_years_df['team_home_abbr'] == 'KC') & (last_5_years_df['score_home'] < last_5_years_df['score_away'])]
chiefs_losses_away = last_5_years_df[(last_5_years_df['team_away_abbr'] == 'KC') & (last_5_years_df['score_away'] < last_5_years_df['score_home'])]

In [ ]:
# Scatter plot for games won and lost by Kansas City Chiefs as the home team
plt.figure(figsize=(10, 6))

# Scatter plot for games won
plt.scatter(chiefs_wins_home['schedule_season'], chiefs_wins_home['score_home'], c='blue', alpha=0.5, label='Chiefs Win - Home')
plt.scatter(chiefs_wins_away['schedule_season'], chiefs_wins_away['score_home'], c='red', alpha=0.5, label='Chiefs Win - Away')
# Scatter plot for games lost
plt.scatter(chiefs_losses_home['schedule_season'], chiefs_losses_home['score_home'], c='green', alpha=0.5, label='Chiefs Loss - Home')
plt.scatter(chiefs_losses_away['schedule_season'], chiefs_losses_away['score_home'], c='yellow', alpha=0.5, label='Chiefs Loss - Away')

plt.title('Games by Kansas City Chiefs as Home Team (Last 5 Years)')
plt.xlabel('Date')
plt.ylabel('Score')
plt.legend()
plt.xticks(chiefs_wins_home['schedule_season'].unique(), rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()
